In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set() 

import pandas as pd
import numpy as np
from scipy import stats as st

import warnings
warnings.filterwarnings('ignore')

import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils import *
from utils_concise import *
from classification_utils import *
from data_generating_funcs import *
import tqdm


In [28]:
X, y, beta_star = generate_floodgate_sims()
n, p = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)
np.where(abs(beta_star)>0)

(array([ 35,  66,  93,  97, 157, 221, 224, 240, 246, 260, 370, 410, 416,
        450, 455, 524, 545, 604, 622, 657, 722, 736, 748, 855, 876, 912,
        914, 938, 974, 991]),)

In [34]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = seed)
trainset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                          torch.tensor(y_train, dtype=torch.float32).view(-1,1))
train_loader = DataLoader(trainset, batch_size=32)

full_nn = NN4vi(p, [p], 1)
#early_stopping = EarlyStopping('val_loss', min_delta=tol)
trainer = pl.Trainer(max_epochs=100)
t0 = time.time()
with io.capture_output() as captured: trainer.fit(full_nn, train_loader, train_loader)
full_time = time.time() - t0
test_loss = nn.MSELoss()(full_nn(X_test), y_test).item()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 1.0 M 
------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.008     Total estimated model params size (MB)


In [35]:
test_loss

1.629307746887207

In [12]:
from sklearn.linear_model import LassoCV

las = LassoCV()
las.fit(X_train, y_train)

LassoCV()

In [13]:
from sklearn.metrics import mean_squared_error as mse

mse(y_test, las.predict(X_test))

1.1603532604217548

In [14]:
Xj_train = dropout(X_train, 7)
Xj_test = dropout(X_test, 7)

In [17]:
# true_vi = 0.1377467
np.sqrt(mse(y_test, las.predict(Xj_test)) - mse(y_test, las.predict(X_test)))

0.1333895400437187

In [19]:
las_j = LassoCV()
las_j.fit(Xj_train, y_train)

np.sqrt(mse(y_test, las_j.predict(Xj_test)) - mse(y_test, las.predict(X_test)))

0.12579783274196202

In [24]:
0.12579783274196202 + se

0.14620991604528438

In [22]:
eps_j = ((y_test.detach().numpy() - las_j.predict(Xj_test)) ** 2)
eps_full = ((y_test.detach().numpy() - las.predict(X_test)) ** 2)
se = np.sqrt(np.var(eps_j - eps_full) / y_test.shape[0])

1.7619969844818115

In [5]:
#hidden_layers = [50]
results = []

n, p = X.shape
full_nn, full_time, test_loss = full_model(X, y, hidden_layers=[p])
for j in [115, 116, 149, 150]:
    print(j)
    vi_lz, lz_time, lz_se = lazy_vi(X, y, j, full_nn)
    vi_dr = dropout_vi(X, y, j, full_nn)
    vi_rt, rt_time, rt_se = retrain_vi(X, y, j, full_nn)
    results.append([j, vi_rt, vi_lz, vi_dr])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 1.0 M 
------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.008     Total estimated model params size (MB)


115
21.544346900318832


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 50.1 K
------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)


116
100.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 50.1 K
------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)


149
35.93813663804626


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 50.1 K
------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)


150
100.0


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 50.1 K
------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)


In [7]:
np.sqrt(results)

array([[10.72380529,  0.11984346,         nan,         nan],
       [10.77032961,         nan,  0.07692173,  0.07944067],
       [12.20655562,  0.17009437,         nan,         nan],
       [12.24744871,  0.28581308,  0.11708268,  0.12144272]])

In [8]:
results

[[115, 0.014362454414367676, -0.0007956027984619141, -0.0005646944046020508],
 [116, -0.014607906341552734, 0.005916953086853027, 0.006310820579528809],
 [149, 0.02893209457397461, -0.002349376678466797, -0.0011376142501831055],
 [150, 0.08168911933898926, 0.013708353042602539, 0.014748334884643555]]